In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

Using TensorFlow backend.
C:\Users\Sraavya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Sraavya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Sraavya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Sraavya\anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
#Intialize the model
model=Sequential()

In [3]:
# Add Convolution Layer
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation="relu"))

In [4]:
#Add Pooling Layer
model.add(MaxPooling2D(pool_size = (2, 2)))

In [5]:
#Add Flattening Layer
model.add(Flatten())

In [6]:
#Add Hidden Layer
model.add(Dense(init="uniform",activation="relu",output_dim=120))

C:\Users\Sraavya\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120, kernel_initializer="uniform")`
  


In [7]:
#Add Output layer
model.add(Dense(init="uniform",activation="sigmoid",output_dim=1))

C:\Users\Sraavya\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [8]:
#Compile the model
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=0.8,
                                   horizontal_flip = True,
                                  vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
x_train = train_datagen.flow_from_directory(r'C:\Users\Sraavya\Desktop\project1\train_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                class_mode = 'binary')
x_test = test_datagen.flow_from_directory(r'C:\Users\Sraavya\Desktop\project1\test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [12]:
print(x_train.class_indices)

{'infected': 0, 'uninfected': 1}


In [13]:
model.fit_generator(x_train,samples_per_epoch = 2000,epochs=50,validation_data=x_test,nb_val_samples=200)

C:\Users\Sraavya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
C:\Users\Sraavya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., steps_per_epoch=62, validation_steps=200)`
  """Entry point for launching an IPython kernel.



Epoch 1/50
62/62 [==============================] - 16s 261ms/step - loss: 0.6027 - accuracy: 0.6507 - val_loss: 0.7833 - val_accuracy: 0.7693
Epoch 2/50
62/62 [==============================] - 17s 276ms/step - loss: 0.3642 - accuracy: 0.8558 - val_loss: 0.4413 - val_accuracy: 0.8328
Epoch 3/50
62/62 [==============================] - 17s 278ms/step - loss: 0.2712 - accuracy: 0.9113 - val_loss: 0.4327 - val_accuracy: 0.7756
Epoch 4/50
62/62 [==============================] - 17s 268ms/step - loss: 0.2896 - accuracy: 0.8851 - val_loss: 0.2654 - val_accuracy: 0.8971
Epoch 5/50
62/62 [==============================] - 18s 284ms/step - loss: 0.2160 - accuracy: 0.9249 - val_loss: 0.4044 - val_accuracy: 0.8331
Epoch 6/50
62/62 [==============================] - 17s 275ms/step - loss: 0.2506 - accuracy: 0.8987 - val_loss: 0.3604 - val_accuracy: 0.8175
Epoch 7/50
62/62 [==============================] - 16s 265ms/step - loss: 0.2071 - accuracy: 0.9183 - val_loss: 0.2135 - val_accuracy: 0.843

In [29]:
model.save('mymodel.h5')

In [30]:
from keras.models import load_model
import numpy as np


In [37]:
import cv2
model=load_model('mymodel.h5')

In [38]:
from skimage.transform import resize

def detect(frame):
    try:
        img = resize(frame,(64,64))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = model.predict(img)
        print(prediction)
        prediction = model.predict_classes(img)
        print(prediction)
    except AttributeError:
        print("shape not found")

In [39]:
frame=cv2.imread(r"C:\Users\Sraavya\Desktop\project1\test_set\infected\12.png")
data=detect(frame)

[[1.6362123e-06]]
[[0]]
